
# PJM Temperature Forecast (Daily)

This notebook builds a daily temperature forecasting model using the PJM-wide averaged series produced in `temp_analysis.ipynb`. The primary target is the next-day average temperature, which can later feed into energy price models.



## Steps
1. Aggregate hourly PJM temperatures to daily averages.
2. Engineer calendar features (day-of-year sine/cosine, month) and lagged temperature metrics (lag 1/7, rolling means).
3. Train/test split (chronological) with an XGBoost regressor.
4. Evaluate forecast accuracy (MAE/RMSE) on a holdout set.
5. Generate a 14-day iterative forecast and export results.


In [1]:

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

try:
    import xgboost as xgb
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'xgboost'])
    import xgboost as xgb

DATA_DIR = Path('output_data')
FORECAST_PATH = Path('output_data/temp_forecast_daily.csv')


## Load hourly PJM temperature series

In [2]:

df_temp = pd.read_csv(DATA_DIR / 'pjm_temperature_hourly.csv', parse_dates=['datetime_beginning_ept'])
df_temp['date'] = df_temp['datetime_beginning_ept'].dt.floor('D')
daily = (
    df_temp.groupby('date')['avg_temp_f']
    .mean()
    .rename('avg_temp_f')
    .to_frame()
)

daily['lag_1'] = daily['avg_temp_f'].shift(1)
daily['lag_7'] = daily['avg_temp_f'].shift(7)
daily['rolling_7'] = daily['avg_temp_f'].rolling(7).mean()
daily['rolling_30'] = daily['avg_temp_f'].rolling(30).mean()
daily['dayofyear'] = daily.index.dayofyear

# Calendar features
annual_period = 365.25
daily['doy_sin'] = np.sin(2 * np.pi * daily['dayofyear'] / annual_period)
daily['doy_cos'] = np.cos(2 * np.pi * daily['dayofyear'] / annual_period)
daily['month'] = daily.index.month

# Target: next-day average temperature
daily['target'] = daily['avg_temp_f'].shift(-1)

daily = daily.dropna().copy()
daily.head()


,avg_temp_f,lag_1,lag_7,rolling_7,rolling_30,dayofyear,doy_sin,doy_cos,month,target
date,,,,,,,,,,
2023-11-04,54.270588,52.507143,65.070588,51.040756,58.334172,308,-0.833183,0.552997,11,54.618681
2023-11-05,54.618681,54.270588,63.167614,49.819480,57.840271,309,-0.823547,0.567248,11,56.053289
2023-11-06,56.053289,54.618681,55.203125,49.940932,57.801199,310,-0.813668,0.581330,11,63.804545
2023-11-07,63.804545,56.053289,44.344118,52.720993,58.067913,311,-0.803548,0.595240,11,57.237121
2023-11-08,57.237121,63.804545,42.618421,54.809379,58.110621,312,-0.793190,0.608975,11,61.441538


## Train/test split and XGBoost training

In [3]:

feature_cols = ['avg_temp_f', 'lag_1', 'lag_7', 'rolling_7', 'rolling_30', 'doy_sin', 'doy_cos', 'month']
X = daily[feature_cols]
y = daily['target']

split_idx = int(len(daily) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

temp_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)
temp_model.fit(X_train, y_train)

train_pred = temp_model.predict(X_train)
test_pred = temp_model.predict(X_test)

metrics = {
    'train_mae': mean_absolute_error(y_train, train_pred),
    'train_rmse': mean_squared_error(y_train, train_pred, squared=False),
    'test_mae': mean_absolute_error(y_test, test_pred),
    'test_rmse': mean_squared_error(y_test, test_pred, squared=False)
}
metrics


/Users/alexj/code/CFT-Energy-Analysis/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/alexj/code/CFT-Energy-Analysis/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'train_mae': np.float64(0.3186690594585716),
 'train_rmse': np.float64(0.40246060109078274),
 'test_mae': np.float64(2.9607466710194066),
 'test_rmse': np.float64(3.758427595525877)}

## Persist metrics

In [4]:

import json
metrics_path = DATA_DIR / 'temp_forecast_metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
metrics_path


PosixPath('output_data/temp_forecast_metrics.json')

## 14-day iterative forecast

In [5]:

forecast_horizon = 14
last_date = daily.index.max()
forecast_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')

history = daily[['avg_temp_f']].copy()
predictions = []

for date in forecast_dates:
    features = {}
    history_index = history.index
    features['avg_temp_f'] = history.iloc[-1]['avg_temp_f']
    features['lag_1'] = history.iloc[-1]['avg_temp_f']
    features['lag_7'] = history.iloc[-7]['avg_temp_f'] if len(history) >= 7 else history.iloc[-1]['avg_temp_f']
    features['rolling_7'] = history['avg_temp_f'].tail(7).mean()
    features['rolling_30'] = history['avg_temp_f'].tail(30).mean()
    dayofyear = date.timetuple().tm_yday
    features['doy_sin'] = np.sin(2 * np.pi * dayofyear / 365.25)
    features['doy_cos'] = np.cos(2 * np.pi * dayofyear / 365.25)
    features['month'] = date.month
    X_future = pd.DataFrame([features])[feature_cols]
    pred = float(temp_model.predict(X_future))
    predictions.append(pred)
    history.loc[date] = pred

forecast_df = pd.DataFrame({
    'date': forecast_dates,
    'predicted_temp_f': predictions
})
forecast_df.head()


/var/folders/_s/ll61fvyd4n5349y83x4ng5400000gn/T/ipykernel_21463/1018918366.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(temp_model.predict(X_future))
/var/folders/_s/ll61fvyd4n5349y83x4ng5400000gn/T/ipykernel_21463/1018918366.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(temp_model.predict(X_future))
/var/folders/_s/ll61fvyd4n5349y83x4ng5400000gn/T/ipykernel_21463/1018918366.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.

,date,predicted_temp_f
0,2024-10-06,64.569458
1,2024-10-07,61.640759
2,2024-10-08,58.533463
3,2024-10-09,56.982010
4,2024-10-10,59.985065


## Export forecast

In [6]:

forecast_df.to_csv(FORECAST_PATH, index=False)
FORECAST_PATH


PosixPath('output_data/temp_forecast_daily.csv')